# 1. Install Dependencies

pip install torch from https://pytorch.org/ based on your specs

In [1]:
#install YOLOv5
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
!cd yolov5 & pip install -r requirements.txt

# 2. Realtime Gauze Detection w/ 2 Webcams

In [3]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import time

# YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../yolov5-master/202318JUL.pt', force_reload=True)
model2 = torch.hub.load('ultralytics/yolov5', 'yolov5x6', force_reload=True)

model.conf = 0.50
#model.max_det = 20
model.classes = [0]
model2.conf = 0.25
#model2.max_det = 20
model2.classes = [0]

onscreenIn, onscreenOut, countIn, countOut, frameCountIn, frameCountOut, startTime, endTime, countPlay = 0, 0, 0, 0, 0, 0, 0, 0, 0
condition = 7
cap1 = cv2.VideoCapture(0)
#Check this
cap2=cap1
#cap2 = cv2.VideoCapture(1)

delay=0.05
time_now=0
while True:
    ret, frame1 = cap1.read()
    ret, frame2 = cap2.read()
    frame1=cv2.resize(frame1,frame1.shape//2)
    frame2=cv2.resize(frame2,frame1.shape//2)

    # Make detections
    if(time_now>delay):
        pic = np.hstack((frame1, frame2))
        results = model(np.hstack((frame1, frame2)))
        results2 = model2(np.hstack((frame1, frame2)))

        a, b, c, d = 0, 0, 0, 0
        for detection in results.xyxy[0]:

            # Gauze detection
            if detection[5].item() in model.classes:
                if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                    a += 1
                elif detection[2].item() > frame1.shape[1]:
                    b += 1

        for detection in results2.xyxy[0]:
            # Human detection
            if detection[5].item() == 0:
                if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                    c += 1
                elif detection[2].item() > frame1.shape[1]:
                    d += 1

        image = np.squeeze(results.render())

        # Adding black border at the bottom
        borderedFrame = cv2.copyMakeBorder(image, 0, 350, 640 - frame1.shape[1], 640 - frame2.shape[1], cv2.BORDER_CONSTANT,
                                           value=[0, 0, 0])

        if c == 0:
            tempcount1 = a
            if tempcount1 != onscreenIn:
                frameCountIn += 1
                if frameCountIn == condition:
                    if tempcount1 > onscreenIn:
                        countIn += tempcount1 - onscreenIn
                    onscreenIn = tempcount1
            else:
                frameCountIn = 0
            cv2.putText(borderedFrame, 'On Screen =' + str(onscreenIn), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Total In =' + str(countIn), (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                        2, cv2.LINE_AA)
        else:
            cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Total In =' + 'Paused', (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Human Detected', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)

        if d == 0:
            tempcount2 = b
            if tempcount2 != onscreenOut:
                frameCountOut += 1
                if frameCountOut == condition:
                    if tempcount2 > onscreenOut:
                        countOut += tempcount2 - onscreenOut
                    onscreenOut = tempcount2
            else:
                frameCountOut = 0
            cv2.putText(borderedFrame, 'On Screen =' + str(onscreenOut), (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Total Out =' + str(countOut), (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
        else:
            cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Total Out =' + 'Paused', (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Human Detected', (840, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # FPS Counter
        endTime = time.time()
        fps = 1 / (endTime - startTime)
        startTime = time.time()
        cv2.putText(borderedFrame, "FPS =" + str(round(fps, 1)), (540, 520), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                    2, cv2.LINE_AA)

        # Gauze In-Play Counter
        countPlay = countIn - onscreenIn - countOut
        cv2.putText(borderedFrame, 'In Play =' + str(countPlay), (360, 650), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4,
                    cv2.LINE_AA)

        cv2.imshow('Gauze', borderedFrame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    else:
        time_now+=time.time()
            

cap1.release()
cap2.release()
cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\user/.cache\torch\hub\master.zip
requirements: Ultralytics requirement ['Pillow>=10.0.1'] not found, attempting AutoUpdate...
requirements:  Command 'pip install --no-cache "Pillow>=10.0.1"  ' returned non-zero exit status 1.
YOLOv5  2023-12-10 Python-3.7.16 torch-1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\user/.cache\torch\hub\master.zip
requirements: Ultralytics requirement ['Pillow>=10.0.1'] not found, attempting AutoUpdate...
requirements:  Command 'pip install --no-cache "Pillow>=10.0.1"  ' returned non-zero exit status 1.
YOLOv5  2023-12-10 Python-3.7.16 torch-1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradien

TypeError: unsupported operand type(s) for //: 'tuple' and 'int'

In [11]:
onscreenIn, onscreenOut, countIn, countOut, frameCountIn, frameCountOut, startTime, endTime, countPlay = 0, 0, 0, 0, 0, 0, 0, 0, 0
condition = 7
cap1 = cv2.VideoCapture(0)
#Check this
cap2=cap1
delay=0.5
time_now=0
while True:
    ret, frame1 = cap1.read()
    ret, frame2 = cap2.read()
    frame1=cv2.resize(frame1,(frame1.shape[1]//2,frame1.shape[0]//2))
    frame2=cv2.resize(frame2,(frame2.shape[1]//2,frame2.shape[0]//2))

    # Make detections
    if(time_now>delay):
        pic = np.hstack((frame1, frame2))
        results = model(np.hstack((frame1, frame2)))
        results2 = model2(np.hstack((frame1, frame2)))

        a, b, c, d = 0, 0, 0, 0
        for detection in results.xyxy[0]:

            # Gauze detection
            if detection[5].item() in model.classes:
                if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                    a += 1
                elif detection[2].item() > frame1.shape[1]:
                    b += 1

        for detection in results2.xyxy[0]:
            # Human detection
            if detection[5].item() == 0:
                if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                    c += 1
                elif detection[2].item() > frame1.shape[1]:
                    d += 1

        image = np.squeeze(results.render())

        # Adding black border at the bottom
        borderedFrame = cv2.copyMakeBorder(image, 0, 350, 640 - frame1.shape[1], 640 - frame2.shape[1], cv2.BORDER_CONSTANT,
                                           value=[0, 0, 0])

        if c == 0:
            tempcount1 = a
            if tempcount1 != onscreenIn:
                frameCountIn += 1
                if frameCountIn == condition:
                    if tempcount1 > onscreenIn:
                        countIn += tempcount1 - onscreenIn
                    onscreenIn = tempcount1
            else:
                frameCountIn = 0
            cv2.putText(borderedFrame, 'On Screen =' + str(onscreenIn), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Total In =' + str(countIn), (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                        2, cv2.LINE_AA)
        else:
            cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Total In =' + 'Paused', (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Human Detected', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)

        if d == 0:
            tempcount2 = b
            if tempcount2 != onscreenOut:
                frameCountOut += 1
                if frameCountOut == condition:
                    if tempcount2 > onscreenOut:
                        countOut += tempcount2 - onscreenOut
                    onscreenOut = tempcount2
            else:
                frameCountOut = 0
            cv2.putText(borderedFrame, 'On Screen =' + str(onscreenOut), (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Total Out =' + str(countOut), (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
        else:
            cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Total Out =' + 'Paused', (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(borderedFrame, 'Human Detected', (840, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # FPS Counter

        cv2.putText(borderedFrame, "FPS =" + str(round(cap1.get(cv2.CAP_PROP_FPS), 1)), (540, 520), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                    2, cv2.LINE_AA)

        # Gauze In-Play Counter
        countPlay = countIn - onscreenIn - countOut
        cv2.putText(borderedFrame, 'In Play =' + str(countPlay), (360, 650), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4,
                    cv2.LINE_AA)

        cv2.imshow('Gauze', borderedFrame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    else:
        time_now+=time.time()
            

cap1.release()
cap2.release()
cv2.destroyAllWindows()

In [26]:
onscreenIn, onscreenOut, countIn, countOut, frameCountIn, frameCountOut, startTime, endTime, countPlay = 0, 0, 0, 0, 0, 0, 0, 0, 0
condition = 7
cap1 = cv2.VideoCapture(0)
#Check this
cap2=cap1
delay=0.5
time_now=0
while True:
    ret, frame1 = cap1.read()
    ret, frame2 = cap2.read()
    cap1.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
    cap1.set(cv2.CAP_PROP_FPS, 60)
    cap2.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
    cap2.set(cv2.CAP_PROP_FPS, 60)
    #frame1=cv2.resize(frame1,(frame1.shape[1]//2,frame1.shape[0]//2))
    #frame2=cv2.resize(frame2,(frame2.shape[1]//2,frame2.shape[0]//2))

    # Make detections
    if(time_now>delay):
        pic = np.hstack((frame1, frame2))
        #results = model(np.hstack((frame1, frame2)))
        #results2 = model2(np.hstack((frame1, frame2)))
        results=frame1
        results=frame2

        endTime = time.time()
        fps = 1 / (endTime - startTime)
        startTime = time.time()
        print(fps)
        cv2.putText(frame1, "FPS =" + str(round(fps, 1)), (540, 520), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                    2, cv2.LINE_AA)
        cv2.imshow('Gauze', frame1)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    else:
        time_now+=time.time()
            

cap1.release()
cap2.release()
cv2.destroyAllWindows()

5.875674234611546e-10
15.653074781492347
14.884344181722044
15.809545348320027
12.543150213822184
15.485250574101558
15.423580850257961
16.085722942162327
15.669038893309574
15.83503161868806
15.169931425595324
11.433076103222781
17.130026015821866
16.52335123167652
15.3834168955918
16.172992982185548
15.783487619477684
11.578159333075691
16.158973667482133
16.1826648918726
15.61705613392312
15.58456073599572
15.342731935004792
9.206595606441077
21.395252985375357
19.640484373975667
15.623571481784996
15.592382071175514
15.152558660428822
8.801744272670613
20.046187963600214
21.65953513351614
15.940226582600797
16.062745098039215
15.663479923518164
10.535302258872347
12.065391176275995
19.589756523607356
21.007442727063278
17.034228438682845
14.980941223533446
10.682695491383658
12.083048371902755
24.67005458309806
18.598533154781435
15.859040964329196
15.83556968429319
10.278217101184335
12.58149029159705
20.946384338793447
20.82139772242134
14.896449830233978
15.536127954484
10.35042